# 数据合并

本笔记本从 preprocess_data.py 中剥离出的数据合并部分

In [ ]:
import pandas as pd

# 合并所有日前数据
print("合并日前数据...")
day_ahead_data = pd.merge(
    df_load[['日期', '时点', '省调负荷(MW)']],
    df_renewable[['日期', '时点', '风电(MW)', '光伏(MW)', '新能源负荷(MW)']],
    on=['日期', '时点'],
    how='outer'
)
day_ahead_data = pd.merge(
    day_ahead_data,
    df_disclosure[['日期', '时点', '非市场化出力(MW)']],
    on=['日期', '时点'],
    how='outer'
)
day_ahead_data = pd.merge(
    day_ahead_data,
    df_tie_line[['日期', '时点', '联络线计划(MW)']],
    on=['日期', '时点'],
    how='outer'
)
day_ahead_data = pd.merge(
    day_ahead_data,
    df_hydro[['日期', '时点', '水电出力(MW)']],
    on=['日期', '时点'],
    how='outer'
)
day_ahead_data = pd.merge(
    day_ahead_data,
    df_price[['日期', '时点', '日前出清价格(元/MWh)']],
    on=['日期', '时点'],
    how='outer'
)

# 添加边界数据类型和在线机组容量
day_ahead_data['边界数据类型'] = '日前'
day_ahead_data['在线机组容量(MW)'] = online_capacity

In [ ]:
# 合并所有实时数据
print("合并实时数据...")
real_time_data = df_actual[['日期', '时点', '省调负荷(MW)', '风电(MW)', '光伏(MW)',
                              '新能源负荷(MW)', '水电出力(MW)', '非市场化出力(MW)']].copy()
real_time_data = pd.merge(
    real_time_data,
    df_tie_line_rt[['日期', '时点', '联络线计划(MW)']],
    on=['日期', '时点'],
    how='left'
)
real_time_data = pd.merge(
    real_time_data,
    df_price[['日期', '时点', '实时出清价格(元/MWh)']],
    on=['日期', '时点'],
    how='left'
)
real_time_data['边界数据类型'] = '实时'

In [ ]:
# 合并日前和实时数据
print("合并所有数据...")
result_df = pd.concat([day_ahead_data, real_time_data], ignore_index=True)

# 添加缺失的列（如果需要）
columns = ['日期', '时点', '边界数据类型', '竞价空间(MW)', '省调负荷(MW)', '风电(MW)',
           '光伏(MW)', '新能源负荷(MW)', '非市场化出力(MW)', '水电出力(MW)',
           '联络线计划(MW)', '在线机组容量(MW)', '日前出清价格(元/MWh)',
           '实时出清价格(元/MWh)', '负荷率(%)']

# 确保所有列都存在
for col in columns:
    if col not in result_df.columns:
        result_df[col] = None

# 重新排序列
result_df = result_df[columns]

In [ ]:
# 排序：先按边界数据类型排序（日前在前，实时在后），再按日期和时点排序
# 这样可以保证：所有日前数据在前（按时点），然后所有实时数据在后（按时点）
result_df['时点_排序'] = pd.to_datetime(result_df['时点'], format='%H:%M', errors='coerce')
result_df['边界数据类型_排序'] = result_df['边界数据类型'].map({'日前': 0, '实时': 1})
result_df = result_df.sort_values(['边界数据类型_排序', '日期', '时点_排序']).reset_index(drop=True)
result_df = result_df.drop(columns=['时点_排序', '边界数据类型_排序'])

print(f"合并完成，总行数: {len(result_df)}")
result_df.head()